<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/B%C4%B0NANCE_A%C5%9EIRI_ALIM_SATIM_EXCEL_TABLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required packages
!pip install python-binance openpyxl

# Import required libraries
import pandas as pd
import numpy as np
import time
from binance.client import Client

# Binance API anahtarlarınızı buraya ekleyin
api_key = 'YOUR_API_KEY'
api_secret = 'YOUR_API_SECRET'

# Binance Client oluşturma (with regional support)
# For Binance US, use tld='us'
client = Client(api_key, api_secret, tld='us')  # Change to 'com' for global

# Fetch valid symbols from Binance
def get_valid_symbols():
    exchange_info = client.get_exchange_info()
    valid_symbols = [symbol['symbol'] for symbol in exchange_info['symbols']]
    return valid_symbols

# Valid symbols list (dynamically fetched)
symbols = get_valid_symbols()

# RSI hesaplama fonksiyonu
def calculate_rsi(data, period=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# MFI hesaplama fonksiyonu
def calculate_mfi(data, period=14):
    typical_price = (data['high'] + data['low'] + data['close']) / 3
    raw_money_flow = typical_price * data['volume']
    positive_flow = (raw_money_flow.where(typical_price.diff() > 0, 0)).rolling(window=period).sum()
    negative_flow = (raw_money_flow.where(typical_price.diff() < 0, 0)).rolling(window=period).sum()
    money_flow_ratio = positive_flow / negative_flow
    mfi = 100 - (100 / (1 + money_flow_ratio))
    return mfi

# Veri çekme ve analiz fonksiyonu
def fetch_and_analyze(symbol):
    try:
        # Fetch historical data
        klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, "1 day ago UTC")

        # Check if data is empty
        if not klines:
            print(f"No data available for {symbol}")
            return None

        # Convert to DataFrame
        data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
        data['close'] = pd.to_numeric(data['close'])
        data['high'] = pd.to_numeric(data['high'])
        data['low'] = pd.to_numeric(data['low'])
        data['volume'] = pd.to_numeric(data['volume'])

        # Check if there's enough data to calculate RSI and MFI
        if len(data) < 14:
            print(f"Not enough data for {symbol} to calculate RSI and MFI")
            return None

        # Calculate RSI and MFI
        data['rsi'] = calculate_rsi(data)
        data['mfi'] = calculate_mfi(data)

        # Son değerleri al
        last_rsi = data['rsi'].iloc[-1]
        last_mfi = data['mfi'].iloc[-1]

        # Alarm koşulları
        rsi_status = "AŞIRI ALIM" if last_rsi > 70 else "AŞIRI SATIM" if last_rsi < 30 else "NORMAL"
        mfi_status = "AŞIRI ALIM" if last_mfi > 80 else "AŞIRI SATIM" if last_mfi < 20 else "NORMAL"

        # Sonuçları bir dictionary olarak döndür
        return {
            'Sembol': symbol,
            'RSI': last_rsi,
            'RSI Durumu': rsi_status,
            'MFI': last_mfi,
            'MFI Durumu': mfi_status
        }
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Tüm sembolleri tarama ve sonuçları toplama
results = []
for symbol in symbols:
    result = fetch_and_analyze(symbol)
    if result:
        results.append(result)
    time.sleep(1)  # API rate limitini aşmamak için bekleme süresi

# Sonuçları bir DataFrame'e dönüştür
results_df = pd.DataFrame(results)

# Excel dosyasına kaydet
excel_file = "binance_analysis_results.xlsx"
results_df.to_excel(excel_file, index=False)

print(f"Sonuçlar '{excel_file}' dosyasına kaydedildi.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.8 MB/s eta 0:00:00
No data available for BTCUSD4
No data available for ETHUSD4
No data available for XRPUSD
No data available for BCHUSD4
No data available for LTCUSD4
No data available for USDTUSD4
No data available for BNBUSD4
No data available for XRPBTC
No data available for BCHBTC
No data available for ADAUSD4
No data available for BATUSD4
No data available for ETCUSD4
No data available for XLMUSD4
No data available for ZRXUSD4
No data available for LINKUSD4
No data available for RVNUSD4
No data available for DASHUSD4
No data available for ZECUSD4
No data available for ALGOUSD4
No data available for IOTAUSD4
No data available for BUSDUSD4
No data available for BTCBUSD
No data available for WAVESUSD4
No data available for ATOMUSD4
No data available for NEOUSD4
No dat